In [ ]:
%gui qt

import sys
import time
import os
import pandas as pd
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QVBoxLayout, QFrame,
    QPushButton, QFileDialog, QWidget, QRubberBand, QLineEdit, QMessageBox
)
from PyQt5.QtCore import Qt, QPoint, QRect
from PyQt5.QtGui import QMouseEvent


class Capture(QWidget):
    def __init__(self, main_window):
        super().__init__()
        self.main = main_window
        self.main.hide()

        self.setMouseTracking(True)
        desk_size = QApplication.desktop()
        self.setGeometry(0, 0, desk_size.width(), desk_size.height())
        self.setWindowFlags(self.windowFlags() | Qt.FramelessWindowHint | Qt.WindowStaysOnTopHint)
        self.setWindowOpacity(0.15)

        self.rubber_band = QRubberBand(QRubberBand.Rectangle, self)
        self.origin = QPoint()

        QApplication.setOverrideCursor(Qt.CrossCursor)
        screen = QApplication.primaryScreen()
        rect = QApplication.desktop().rect()

        time.sleep(0.31)  # short delay to ensure capture after UI disappears
        self.imgmap = screen.grabWindow(
            QApplication.desktop().winId(),
            rect.x(), rect.y(), rect.width(), rect.height()
        )

    def mousePressEvent(self, event: QMouseEvent) -> None:
        if event.button() == Qt.LeftButton:
            self.origin = event.pos()
            self.rubber_band.setGeometry(QRect(self.origin, event.pos()).normalized())
            self.rubber_band.show()

    def mouseMoveEvent(self, event: QMouseEvent) -> None:
        if not self.origin.isNull():
            self.rubber_band.setGeometry(QRect(self.origin, event.pos()).normalized())

    def mouseReleaseEvent(self, event: QMouseEvent) -> None:
        if event.button() == Qt.LeftButton:
            self.rubber_band.hide()

            rect = self.rubber_band.geometry()
            self.imgmap = self.imgmap.copy(rect)
            QApplication.restoreOverrideCursor()

            clipboard = QApplication.clipboard()
            clipboard.setPixmap(self.imgmap)

            self.main.label.setPixmap(self.imgmap)
            self.main.label.adjustSize()

            # Resize window to image size + padding
            img_size = self.imgmap.size()
            padding_height = 150  # for buttons and input
            padding_width = 50

            new_width = img_size.width() + padding_width
            new_height = img_size.height() + padding_height
            self.main.resize(new_width, new_height)

            self.main.float_input.setVisible(True)
            self.main.btn_save.setVisible(True)
            self.main.show()

            self.close()


class ScreenRegionSelector(QMainWindow):
    def __init__(self):
        super().__init__(None)
        self.setWindowTitle("Screen Capturer")
        self.setMinimumSize(600, 400)

        frame = QFrame()
        frame.setContentsMargins(0, 0, 0, 0)
        lay = QVBoxLayout(frame)
        lay.setAlignment(Qt.AlignCenter)
        lay.setContentsMargins(10, 10, 10, 10)

        self.label = QLabel()
        self.label.setAlignment(Qt.AlignCenter)
        self.label.setStyleSheet("background-color: #222222;")

        self.btn_capture = QPushButton("Capture")
        self.btn_capture.clicked.connect(self.capture)

        self.float_input = QLineEdit()
        self.float_input.setPlaceholderText("Enter float value for this capture")
        self.float_input.setVisible(False)

        self.btn_save = QPushButton("Save")
        self.btn_save.clicked.connect(self.save)
        self.btn_save.setVisible(False)

        lay.addWidget(self.label)
        lay.addWidget(self.btn_capture)
        lay.addWidget(self.float_input)
        lay.addWidget(self.btn_save)

        self.setCentralWidget(frame)

    def capture(self):
        self.capturer = Capture(self)
        self.capturer.show()

    def save(self):
        try:
            float_value = float(self.float_input.text())
        except ValueError:
            QMessageBox.warning(self, "Invalid Input", "Please enter a valid float value.")
            return

        timestamp = time.strftime("%Y_%m_%d_%H_%M_%S")
        default_name = f"pokemon_{timestamp}.png"

        file_name, _ = QFileDialog.getSaveFileName(self, "Save Image", default_name, "Image files (*.png *.jpg *.bmp)")
        if file_name:
            self.capturer.imgmap.save(file_name)

            # Save to CSV in format: filename,,,float
            csv_path = "pokemon_data.csv"
            entry = f"{os.path.basename(file_name)},,,{float_value}\n"

            with open(csv_path, "a") as f:
                f.write(entry)

            # Reset UI
            self.float_input.clear()
            self.float_input.setVisible(False)
            self.btn_save.setVisible(False)
            self.label.clear()


# Create app and run
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)

app.setStyleSheet("""
QFrame {
    background-color: #3f3f3f;
}
QPushButton {
    border-radius: 5px;
    background-color: rgb(60, 90, 255);
    padding: 10px;
    color: white;
    font-weight: bold;
    font-family: Arial;
    font-size: 12px;
}
QPushButton::hover {
    background-color: rgb(60, 20, 255)
}
QLineEdit {
    padding: 8px;
    font-size: 12px;
    font-family: Arial;
    background-color: white;
}
QLabel {
    background-color: #222;
    border: 1px solid #666;
}
""")

selector = ScreenRegionSelector()
selector.show()

ImportError: DLL load failed while importing onnxruntime_pybind11_state: A dynamic link library (DLL) initialization routine failed.